In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import sqlalchemy

In [2]:
from api_keys import mysql_key as sqlkey

In [3]:
def recreate_wiki(cities):
  # empty list that will be filled with one dictionary of information per city
  list_for_df = []

  # begin a for loop to create a dictionary of information for each city
  for city in cities:
    # we can use the universal nature of wikipedias urls to our advantage here
    # all of the urls are the same besides the city name
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
    # not all of the wikipedia pages contain elevation, look at Hamburg
    # the if clause means that our code can continue and won't stop at this hurdle
    # if soup.select_one('.infobox-label:-soup-contains("Elevation")'):
        # response_dict['elevation'] = soup.select_one('.infobox-label:-soup-contains("Elevation")').find_next(class_='infobox-data').get_text()
    # response_dict['website'] = soup.select_one('.infobox-label:-soup-contains("Website")').find_next(class_='infobox-data').get_text()
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)

  # make the DataFrame
  cities_df = pd.DataFrame(list_for_df)

  # fixing latitude
  cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing longitude
  cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing elevation
  # cities_df.insert(4, 'elevation_in_meters', cities_df['elevation'].str.split('m').str[0].str.strip())

  # return the DataFrame
  return cities_df

In [4]:
cities = ['Berlin', 'London', 'Manchester', 'Istanbul']

In [ ]:
# empty list that will be filled with one dictionary of information per city
list_for_df = []

  # begin a for loop to create a dictionary of information for each city
for city in cities:
    # we can use the universal nature of wikipedias urls to our advantage here
    # all of the urls are the same besides the city name
    url = f'https://en.wikipedia.org/wiki/{city}'

    # here we make our soup for the city
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
    response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
    response_dict['city'] = soup.select(".firstHeading")[0].get_text()
    response_dict['country'] = soup.select(".infobox-data")[0].get_text()
    response_dict['latitude'] = soup.select(".latitude")[0].get_text()
    response_dict['longitude'] = soup.select(".longitude")[0].get_text()
    # not all of the wikipedia pages contain elevation, look at Hamburg
    # the if clause means that our code can continue and won't stop at this hurdle
    # if soup.select_one('.infobox-label:-soup-contains("Elevation")'):
        # response_dict['elevation'] = soup.select_one('.infobox-label:-soup-contains("Elevation")').find_next(class_='infobox-data').get_text()
    # response_dict['website'] = soup.select_one('.infobox-label:-soup-contains("Website")').find_next(class_='infobox-data').get_text()
    if soup.select_one('th.infobox-header:-soup-contains("Population")'):
        response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

    # add our dictionary for the city to list_for_df
    list_for_df.append(response_dict)

# make the DataFrame
cities_df = pd.DataFrame(list_for_df)

# fixing latitude
cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
# fixing longitude
cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)

In [6]:
cities_df

,city,country,latitude,longitude,population
0,Berlin,Germany,52.3112,13.2418,"3,850,809"
1,London,United Kingdom,51.3026,0.739,"8,799,800"
2,Manchester,United Kingdom,53.2846,2.1443,"551,938"
3,Istanbul,Turkey,41.0049,28.5718,"15,907,951"


In [7]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city        4 non-null      object
 1   country     4 non-null      object
 2   latitude    4 non-null      object
 3   longitude   4 non-null      object
 4   population  4 non-null      object
dtypes: object(5)
memory usage: 292.0+ bytes


In [8]:
id = [1,2,3,4]

In [9]:
cities_df['city_id'] = id

In [10]:
cities_df

,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,"3,850,809",1
1,London,United Kingdom,51.3026,0.739,"8,799,800",2
2,Manchester,United Kingdom,53.2846,2.1443,"551,938",3
3,Istanbul,Turkey,41.0049,28.5718,"15,907,951",4


In [11]:
#clean data and change data types 
cities_df['population'] = cities_df['population'].str.replace(',', '', regex=False)
cities_df['population'] = cities_df['population'].astype(int)
cities_df['latitude'] = cities_df['latitude'].astype(float)
cities_df['longitude'] = cities_df['longitude'].astype(float)

In [12]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        4 non-null      object 
 1   country     4 non-null      object 
 2   latitude    4 non-null      float64
 3   longitude   4 non-null      float64
 4   population  4 non-null      int64  
 5   city_id     4 non-null      int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 324.0+ bytes


In [13]:
cities_df

,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,3850809,1
1,London,United Kingdom,51.3026,0.7390,8799800,2
2,Manchester,United Kingdom,53.2846,2.1443,551938,3
3,Istanbul,Turkey,41.0049,28.5718,15907951,4


In [14]:
#export to a .csv file which can be used later as additional data source
cities_df.to_csv('cities.csv', index=False)

## Connect to the MySQL database

In [15]:
cities_df = pd.read_csv('cities.csv')


In [16]:
cities_df

,city,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,3850809,1
1,London,United Kingdom,51.3026,0.7390,8799800,2
2,Manchester,United Kingdom,53.2846,2.1443,551938,3
3,Istanbul,Turkey,41.0049,28.5718,15907951,4


In [19]:
#make a connection to our local database
host = '127.0.0.1'
schema = 'gans_local' # The name of your database (schema) might be different
user = 'root'
password = 'sqlkey'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
#push data to mysql table in local database
cities_df.to_sql('cities',con=con,if_exists='append',index=False)

In [ ]:
cities_df
print(cities_df.info())

## Connect MySQL to AWS

In [21]:
#make a connection to AWS 
host = 'wbs-project5-db.csv6puy3yaat.eu-north-1.rds.amazonaws.com'
schema = 'gans_aws' # The name of your database (schema) might be different
user = 'admin'
password = 'awskey'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cities_df.to_sql('cities',con=con,if_exists='append',index=False)

In [25]:
# read it and check it
cities_df = pd.read_sql('cities', con=con)
cities_df

,city_id,city,country,latitude,longitude,population
0,1,Berlin,Germany,52.3112,13.2418,3850810.0
1,2,London,United Kingdom,51.3026,0.7390,8799800.0
2,3,Manchester,United Kingdom,53.2846,2.1443,551938.0
3,4,Istanbul,Turkey,41.0049,28.5718,15908000.0
